In [1]:
import os
import datetime
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
import matplotlib.pyplot as plt
from PIL import Image

root_dir = '/glade/u/home/hongli/scratch/2020_04_21nldas_gmet'   
nldas_dir = os.path.join(root_dir,'data/nldas_daily_utc')
stnlist_dir = os.path.join(root_dir, 'scripts/step1_sample_stnlist_uniform')
stnlist_name_base = 'stnlist'

time_format='%Y-%m-%d'
start_yr = 2015
end_yr = 2016

time_format='%Y-%m-%d'
extract_start_date = '2015-01-01'
extract_end_date   = '2016-12-31'

outfolder = 'scripts/step14_check_prepare_stndata_uniform'
if not os.path.exists(os.path.join(root_dir, outfolder)):
    os.makedirs(os.path.join(root_dir, outfolder))

# ============================================================================================================
# read historical nldas 
print('Read nldas data')
for yr in range(start_yr, end_yr+1):
    
    nldas_file = 'NLDAS_'+str(yr)+'.nc'
    nldas_path = os.path.join(nldas_dir, nldas_file)
    
    f_nldas = xr.open_dataset(nldas_path)
    if yr == start_yr:
        prcp_avg = f_nldas['prcp_avg'].values[:] # (time, lat, lon). unit: kg/m^2 = mm
        tair_min = f_nldas['tair_min'].values[:] # (time, lat, lon). unit: K
        tair_max = f_nldas['tair_max'].values[:]
        time = pd.to_datetime(f_nldas['time'].values[:]).strftime(time_format)
    else:
        prcp_avg = np.concatenate((prcp_avg, f_nldas['prcp_avg'].values[:]), axis = 0)
        tair_min = np.concatenate((tair_min, f_nldas['tair_min'].values[:]), axis = 0)
        tair_max = np.concatenate((tair_max, f_nldas['tair_max'].values[:]), axis = 0)
        time = np.concatenate((time, pd.to_datetime(f_nldas['time'].values[:]).strftime(time_format)), axis = 0)
    f_nldas.close()
    
prcp_sum = np.multiply(prcp_avg, 24.0) #mm/hr to mm/day
tair_min = np.subtract(tair_min, 273.15)
tair_max = np.subtract(tair_max, 273.15)

# nldas mask on the time dimension
time_obj = np.asarray([datetime.datetime.strptime(t, time_format) for t in time])
start_date_obj = datetime.datetime.strptime(extract_start_date, time_format)
end_date_obj = datetime.datetime.strptime(extract_end_date, time_format)
nldas_mask  = (time_obj >= start_date_obj) & (time_obj <= end_date_obj) 
t = time_obj[nldas_mask]

# ============================================================================================================
# plot station time series one-by-one
stnlist_files = [f for f in os.listdir(stnlist_dir) if stnlist_name_base in f]
stnlist_files = sorted(stnlist_files)

# for stnlist_file in stnlist_files:
for stnlist_file in stnlist_files[0:1]:
    
    # create sub-outfolder
    sub_folder = 'stndata_'+(stnlist_file.split('.')[0].split('_')[1])
    if not os.path.exists(os.path.join(root_dir, outfolder, sub_folder)):
        os.makedirs(os.path.join(root_dir, outfolder, sub_folder)) 
    print(sub_folder)

    # read selected stn list 
    stn_ids = np.loadtxt(os.path.join(stnlist_dir,stnlist_file), skiprows=2, usecols=[0], delimiter=',', dtype='str') # STA_ID[0], LAT[1], LON[2] ELEV[3], SLP_N[4], SLP_E[5], STA_NAME[6]
    stnlist = np.loadtxt(os.path.join(stnlist_dir,stnlist_file), skiprows=2, usecols=[1,2,3,4,5], delimiter=',') 
    stn_num = len(stn_ids)
    
    #====================================================================================#
    # plot each station
    fig_file_summary = []
    for i in range(stn_num):
        stn_id = stn_ids[i]    
        stn_lat_id = int(stn_id[3:3+3]) #start from zero
        stn_lon_id = int(stn_id[9:9+3])
       
        # plot time series to avoid empty values
        fig, ax1 = plt.subplots(constrained_layout=True)
        fig.set_figwidth(3.5*2) #190mm
        fig.set_figheight(3.5) #5.61 heigh/width=3/4
        fig.suptitle(stn_id, fontsize='medium', fontweight='semibold', color='g')

        color = 'tab:red'
        ax1.set_xlabel('time')
        ax1.set_ylabel('T (degC)', color=color)
        lns1=ax1.plot(t, tair_min[nldas_mask,stn_lat_id, stn_lon_id], color='b', linewidth=1, label='tmin')
        lns2=ax1.plot(t, tair_max[nldas_mask,stn_lat_id, stn_lon_id], color='r', linewidth=1, label='tmax')
        ax1.tick_params(axis='y', labelcolor=color)

        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
        color = 'k'
        ax2.set_ylabel('P (mm/d)', color=color)  # we already handled the x-label with ax1
        lns3=ax2.plot(t, prcp_sum[nldas_mask,stn_lat_id, stn_lon_id], color='k', linewidth=1, label='pcp')
        ax2.tick_params(axis='y', labelcolor=color)

        # added these three lines
        lns = lns1+lns2+lns3
        labs = [l.get_label() for l in lns]
        ax1.legend(lns, labs, loc='upper right', fontsize='small', framealpha=0.5)

#         fig.tight_layout()  # otherwise the right y-label is slightly clipped
#         plt.show()

        output_filename = stn_id+'.png'
        fig.savefig(os.path.join(root_dir, outfolder, sub_folder, output_filename))
        plt.close(fig)        

        fig_file_summary.append(os.path.join(root_dir, outfolder, sub_folder, output_filename))
    
#     #====================================================================================#
#     # save all station figures as one
#     widths = []
#     heights = []
#     for fig_file in fig_file_summary:
#         im = Image.open(fig_file)
#         widths.append(im.width)
#         heights.append(im.height)

#     max_width = max(widths)
#     total_height = sum(heights)
#     new_im = Image.new('RGB', (max_width, total_height))

#     x_offset = 0
#     for fig_file in fig_file_summary:
#         im = Image.open(fig_file)    
#         new_im.paste(im, (0,x_offset))
#         x_offset += im.size[1]
#     new_im.save(os.path.join(root_dir, outfolder, sub_folder+'.png'))

print('Done')


/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


Read nldas data
stndata_00810grids


/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/pandas/plotting/_matplotlib/converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


Done


In [4]:
stnlist_files

[]